# 🛡️ ATT&CK Ground Segment Threat Graph — Interactive Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrymaesen/attack-gseg/blob/main/demo.ipynb)

This notebook demonstrates the full **attack-gseg** pipeline:

1. **Ingest** — Download & parse MITRE ATT&CK STIX data
2. **Build Graph** — Construct a NetworkX knowledge graph
3. **Retrieve** — BM25 keyword search over techniques
4. **Rerank** — Semantic reranking with sentence-transformers
5. **API** — Query the FastAPI endpoint

> **Runtime:** Select *Runtime → Change runtime type → CPU* (GPU not required).

## 1. Installation

Clone the repository and install dependencies.

In [ ]:
import os

# Clone the repository (skip if already cloned)
if not os.path.isdir("attack-gseg"):
    !git clone https://github.com/thierrymaesen/attack-gseg.git

os.chdir("attack-gseg")
print(f"Working directory: {os.getcwd()}")

# Install the package in editable mode
!pip install -e . --quiet
print("\nInstallation complete.")

## 2. Sprint 1 — Ingest ATT&CK STIX Data

Download the official MITRE ATT&CK Enterprise STIX bundle and parse techniques, mitigations, and their relationships.

In [ ]:
from gseg.ingest_attack import (
    download_attack_stix,
    parse_techniques,
    parse_mitigations,
    parse_relations,
    save_json,
)
from pathlib import Path
import json
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)-8s | %(message)s", force=True)

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Download STIX bundle
stix_path = data_dir / "enterprise-attack.json"
download_attack_stix(stix_path)

# Load and parse
with open(stix_path, "r") as f:
    stix_data = json.load(f)

techniques = parse_techniques(stix_data)
mitigations = parse_mitigations(stix_data)
relations = parse_relations(stix_data, techniques, mitigations)

# Save structured JSON
save_json(techniques, data_dir / "techniques.json")
save_json(mitigations, data_dir / "mitigations.json")
save_json(relations, data_dir / "relations.json")

print(f"\nTechniques : {len(techniques)}")
print(f"Mitigations: {len(mitigations)}")
print(f"Relations  : {len(relations)}")

## 3. Sprint 2 — Build Knowledge Graph

Construct a directed NetworkX graph and a BM25 text index from the parsed data.

In [ ]:
from gseg.build_graph import (
    load_json,
    build_graph,
    build_text_index,
    compute_graph_stats,
    save_graph,
    save_text_index,
    display_stats,
)

techniques_data = load_json(data_dir / "techniques.json")
mitigations_data = load_json(data_dir / "mitigations.json")
relations_data = load_json(data_dir / "relations.json")

# Build graph
graph = build_graph(techniques_data, mitigations_data, relations_data)
text_index = build_text_index(graph)

# Save artefacts
save_graph(graph, data_dir / "attack_graph.gpickle")
save_text_index(text_index, data_dir / "text_index.json")

# Display statistics
stats = compute_graph_stats(graph)
display_stats(stats)

## 4. Sprint 3 — BM25 Retrieval

Search for ATT&CK techniques using keyword-based BM25 retrieval.

In [ ]:
from gseg.retrieve import RetrieverBM25

retriever = RetrieverBM25(
    graph_path=data_dir / "attack_graph.gpickle",
    text_index_path=data_dir / "text_index.json",
)

# Search example
query = "SSH lateral movement"
hits = retriever.search(query, top_k=5)

print(f"Query: {query!r}\n")
for i, hit in enumerate(hits, 1):
    tactics = ", ".join(hit.tactics) or "n/a"
    print(f"  {i}) {hit.technique_id} - {hit.name}  (score={hit.bm25_score:.2f})")
    print(f"     Tactics: [{tactics}]")

# Show mitigations for top result
if hits:
    top = hits[0]
    mits = retriever.get_mitigations(top.technique_id)
    print(f"\nMitigations for {top.technique_id} ({top.name}):")
    for m in mits[:5]:
        print(f"  - {m['mitigation_id']}: {m['name']}")

## 5. Sprint 4 — Semantic Reranking

Rerank BM25 candidates using `all-MiniLM-L6-v2` sentence embeddings for better semantic relevance.

In [ ]:
from gseg.rank import Reranker, combine_retrieval_rerank

reranker = Reranker(model_name="all-MiniLM-L6-v2", device="cpu")

# Full pipeline: BM25 retrieval + semantic reranking
query = "process injection to evade detection"
result = combine_retrieval_rerank(
    retriever=retriever,
    reranker=reranker,
    query=query,
    bm25_k=20,
    final_k=5,
)

print(f"Query: {result['query']!r}")
print(f"BM25 candidates: {result['bm25_candidates']}")
print(f"Latency: {result['latency_ms']:.1f} ms\n")

for i, r in enumerate(result["results"], 1):
    tactics = ", ".join(r.get("tactics", []))
    print(f"  {i}) {r['technique_id']} - {r['name']}")
    print(f"     rerank={r['rerank_score']:.4f}  bm25={r['bm25_score']:.2f}  was_rank={r['original_rank']}")
    print(f"     Tactics: [{tactics}]")

## 6. Sprint 5 — FastAPI (Background Server)

Start the API server in a background thread and query it with `requests`.

In [ ]:
import threading
import time
import uvicorn
import requests

from gseg.api import app

# Start the API in a background thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(15)  # Wait for models to load
print("API server started on http://localhost:8000")

In [ ]:
# Health check
resp = requests.get("http://localhost:8000/health")
print(f"Health: {resp.json()}")

In [ ]:
# Map a security event to ATT&CK techniques
payload = {
    "text": "Detected SSH lateral movement to ground station 10.0.0.5",
    "top_k": 5,
    "include_mitigations": True,
}
resp = requests.post("http://localhost:8000/map_event", json=payload)
data = resp.json()

print(f"Query: {data['query']}")
print(f"Latency: {data['latency_ms']:.1f} ms\n")

for i, t in enumerate(data["results"], 1):
    print(f"  {i}) {t['technique_id']} - {t['name']}")
    print(f"     rerank={t['rerank_score']:.4f}  bm25={t['bm25_score']:.2f}")
    if t.get("mitigations"):
        for m in t["mitigations"][:3]:
            print(f"       -> {m['mitigation_id']}: {m['name']}")

In [ ]:
# List techniques (paginated)
resp = requests.get("http://localhost:8000/techniques", params={"limit": 5, "offset": 0})
for t in resp.json():
    print(f"  {t['technique_id']}: {t['name']}  tactics={t['tactics']}")

## 7. Try Your Own Queries

Modify the `query` variable below and re-run the cell to test different security events.

In [ ]:
# ---- Change this query to test different events ----
query = "phishing email with malicious attachment"
# -----------------------------------------------------

payload = {"text": query, "top_k": 5, "include_mitigations": True}
resp = requests.post("http://localhost:8000/map_event", json=payload)
data = resp.json()

print(f"Query: {data['query']}")
print(f"Latency: {data['latency_ms']:.1f} ms\n")

for i, t in enumerate(data["results"], 1):
    print(f"  {i}) {t['technique_id']} - {t['name']}")
    print(f"     rerank={t['rerank_score']:.4f}  tactics={t['tactics']}")
    if t.get("mitigations"):
        for m in t["mitigations"][:3]:
            print(f"       -> {m['mitigation_id']}: {m['name']}")
    print()

---

**License:** MIT — MITRE ATT&CK® is a registered trademark of The MITRE Corporation.